# SQL (SQLAlchemy)

> [Structured Query Language (SQL)](https://en.wikipedia.org/wiki/SQL)은 프로그래밍에 사용되는 도메인 특화 언어로, 관계형 데이터베이스 관리 시스템(RDBMS)에서 데이터를 관리하거나 관계형 데이터 스트림 관리 시스템(RDSMS)에서 스트림 처리를 위해 설계되었습니다. 특히 엔티티와 변수 간의 관계를 포함하는 구조화된 데이터를 다루는 데 유용합니다.

> [SQLAlchemy](https://github.com/sqlalchemy/sqlalchemy)는 MIT 라이선스에 따라 배포되는 Python 프로그래밍 언어용 오픈 소스 `SQL` 툴킷이자 객체 관계 매퍼(ORM)입니다.

이 노트북에서는 `SQLAlchemy`가 지원하는 모든 데이터베이스에 채팅 기록을 저장할 수 있는 `SQLChatMessageHistory` 클래스에 대해 설명합니다.

`SQLite` 이외의 데이터베이스와 함께 사용하려면 해당 데이터베이스 드라이버를 설치해야 합니다.


## 설치

`langchain-community` 패키지에 포함되어 있으므로, 해당 패키지를 설치해야 합니다. 또한 `SQLAlchemy` 패키지도 설치해야 합니다.


In [ ]:
%pip install -qU langchain-community SQLAlchemy langchain-openai

또한 최고 수준의 관찰 가능성을 위해 [LangSmith](https://smith.langchain.com/)를 설정하는 것이 도움이 될 수 있습니다(필수는 아님).


In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"  # LANGCHAIN_TRACING_V2 환경 변수를 "true"로 설정합니다.
# os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN_API_KEY를 설정합니다."

In [9]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## 사용방법

storage를 사용하려면 다음 2가지만 제공하면 됩니다:

1. Session Id - 사용자 이름, 이메일, 채팅 ID 등과 같은 세션의 고유 식별자입니다.

2. Connection string - 데이터베이스 연결을 지정하는 문자열입니다. 이 문자열은 SQLAlchemy의 create_engine 함수에 전달됩니다.


- `SQLChatMessageHistory` 클래스를 사용하여 대화 메시지 기록을 관리합니다.
- `session_id`와 `connection_string`을 인자로 전달하여 `SQLChatMessageHistory` 인스턴스를 생성합니다.
- `add_user_message` 메서드를 사용하여 사용자 메시지를 대화 기록에 추가합니다.
- `add_ai_message` 메서드를 사용하여 AI 메시지를 대화 기록에 추가합니다.


In [1]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 문자열을 전달합니다.
chat_message_history = SQLChatMessageHistory(
    session_id="sql_chat_history", connection_string="sqlite:///sqlite.db"
)

# 사용자 메시지를 추가합니다.
chat_message_history.add_user_message(
    "Hi! My name is Teddy. I am a AI programmer. Nice to meet you!"
)
# AI 메시지를 추가합니다.
chat_message_history.add_ai_message("Hi Teddy! Nice to meet you too!")

- 저장된 대화내용을 확인합니다. `chat_message_history.messages`


In [2]:
# 채팅 메시지 기록의 메시지들
chat_message_history.messages

[HumanMessage(content='Hi! My name is Teddy. I am a AI programmer. Nice to meet you!'),
 AIMessage(content='Hi Teddy! Nice to meet you too!')]

## Chain 에 적용

우리는 이 메시지 기록 클래스를 [LCEL Runnables](/docs/expression_language/how_to/message_history)와 쉽게 결합할 수 있습니다.


In [3]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

- `ChatPromptTemplate`을 사용하여 프롬프트 템플릿을 정의합니다.
  - "system" 메시지: "You are a helpful assistant."
  - `MessagesPlaceholder`를 사용하여 "history" 변수를 프롬프트에 포함시킵니다.
  - "human" 메시지: "{question}"
- 프롬프트 템플릿과 `ChatOpenAI` 모델을 파이프라인으로 연결하여 `chain`을 생성합니다.


In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        # 시스템 메시지를 설정하여 어시스턴트의 역할을 정의합니다.
        ("system", "You are a helpful assistant."),
        # 이전 대화 내용을 포함하기 위한 플레이스홀더를 추가합니다.
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),  # 사용자의 질문을 입력받는 메시지를 설정합니다.
    ]
)

chain = (
    prompt | ChatOpenAI()
)  # 프롬프트와 ChatOpenAI 모델을 연결하여 체인을 생성합니다.

- `RunnableWithMessageHistory` 클래스를 사용하여 `chain`과 메시지 기록을 연결합니다.
- `lambda` 함수를 사용하여 `session_id`에 따라 `SQLChatMessageHistory` 인스턴스를 생성합니다.
  - `SQLChatMessageHistory`는 SQLite 데이터베이스(sqlite.db)에 연결되어 채팅 메시지 기록을 저장합니다.
- `input_messages_key`를 "question"으로 설정하여 입력 메시지의 키를 지정합니다.
- `history_messages_key`를 "history"로 설정하여 기록 메시지의 키를 지정합니다.


In [5]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: SQLChatMessageHistory(
        session_id=session_id, connection_string="sqlite:///sqlite.db"
    ),  # session_id를 기반으로 SQLChatMessageHistory 객체를 생성하는 람다 함수
    input_messages_key="question",  # 입력 메시지의 키를 "question"으로 설정
    history_messages_key="history",  # 대화 기록 메시지의 키를 "history"로 설정
)

- `config` 딕셔너리를 정의하고, `"configurable"` 키 아래에 `"session_id"` 키-값 쌍을 설정합니다.


In [7]:
# 세션 ID를 구성하는 곳입니다.
config = {"configurable": {"session_id": "sql_chat_history"}}

질문에 이름을 물어보는 질문을 해보겠습니다. 이전에 저장한 대화가 있다면, 올바르게 답할 것입니다.

- `chain_with_history` 객체의 `invoke` 메서드를 호출하여 질문에 대한 답변을 생성합니다.
- `invoke` 메서드에는 질문 딕셔너리와 `config` 설정이 전달됩니다.


In [8]:
# 질문 "Whats my name"과 설정을 사용하여 대화 기록이 있는 체인을 호출합니다.
chain_with_history.invoke({"question": "Whats my name?"}, config=config)

AIMessage(content='Your name is Teddy.', response_metadata={'finish_reason': 'stop', 'logprobs': None})